# SEIR, birth, death, periodic, waning and introductions
{func}`.SEIR_Birth_Death_Periodic_Waning_Intro`

This model includes relatively more detail than the other pre-defined models provided and may serve as a template for more complex models.

In addition to the processes of births, deaths and seasonal driving, we have included (i) immune waning, which transitions recovered individuals back to susceptible at a rate $w$ and (ii) an external force of infection, which allows individuals to be infected from outside the population (analogous to case importation) at a rate $\epsilon$.

$$\begin{aligned}
\frac{\mathrm{d}S}{\mathrm{d}t} &= - \frac{\beta(t) SI}{N} + w R + \mu N - \epsilon S - \mu S\\
\frac{\mathrm{d}E}{\mathrm{d}t} &= \frac{\beta(t) SI}{N} + \epsilon S - \alpha E - \mu E \\
\frac{\mathrm{d}I}{\mathrm{d}t} &= \alpha E - \gamma I - \mu I \\
\frac{\mathrm{d}R}{\mathrm{d}t} &= \gamma I - w R - \mu R \\
\beta(t) &= \beta_0 \left(1+\delta \cos \left(\frac{2 \pi t}{P} \right) \right)
\end{aligned}$$

We solve this set of equations deterministically:

In [36]:
from pygom import common_models
import matplotlib.pyplot as plt
import numpy as np
import math

# Set up PyGOM object
n_pop=1e5
mu=0.01/365
alpha=1/2
gamma=1/4
epsilon=100/(365*n_pop)  # approximately 100*n_sus*365/(365*n_pop)=100*frac_sus~30 infections from external sources per year
w=1/(2*365)              # waning rate, immunity lasts ~ 2 years.
beta0=1
delta=0.2
period=365

ode = common_models.SEIR_Birth_Death_Periodic_Waning_Intro({'mu':mu,
                                                            'alpha':alpha,
                                                            'gamma':gamma,
                                                            'epsilon':epsilon,
                                                            'w':w,
                                                            'beta0':beta0,
                                                            'delta':delta,
                                                            'period':period,
                                                            'N':n_pop})

# Time range and increments
tmax=365*20                           # maximum time over which to run solver
dt=0.1                                # timestep
n_timestep=math.ceil(tmax/dt)         # number of iterations
t = np.linspace(0, tmax, n_timestep)  # times at which solution will be evaluated

# Initial conditions
x0 = [n_pop, 0, 0, 0, t[0]]

ode.initial_values = (x0, t[0])

solution=ode.solve_determ(t[1::])

Plotting the infection prevalence reveals that the system eventually reaches a state of annual epidemics.

In [ ]:
f, ax = plt.subplots(figsize=(10, 2))

ax.set_xlabel("Time")
ax.set_ylabel("Infection prevalence")
ax.plot(t[30000:]/365, solution[30000:,2])
plt.show()